In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo

$$
\begin{split}
J_{\kappa \lambda} & = \sum_{i=1}^N  \sum_{\mu}^{M} \sum_{\nu}^{M} c_{i, \mu} c_{i, \nu} [\mu \nu | \kappa \lambda]\\
J_{\kappa \lambda} & = Tr[\bf{C}^T  \bf{G}_{\kappa \lambda} \bf{C}] \\
J_{\kappa \lambda} & = \sum_{i=1}^N  [\phi_i \phi_i | \kappa \lambda]
\end{split}
$$

where $[\bf{G}_{\kappa \lambda}]_{\mu \nu} = [\mu \nu | \kappa \lambda]$ the matrix spanned by $\mu, \nu$ for fixed index pair $\kappa \lambda$

In [2]:
def build_J(Fouridx,C,N):
    J = np.zeros(C.shape)
    for kappa in range(0,Fouridx.shape[0]):
        for lamda in range(0,Fouridx.shape[0]):
            J[kappa][lamda] = np.trace(C[:,0:N].T@Fouridx[:,:,kappa,lamda]@C[:,0:N])

    return 2*J # needs factor 2 because its spin restricted

$$
\begin{split}
J_{\kappa \lambda} & = \sum_{i=1}^N  \sum_{\mu}^{M} \sum_{\nu}^{M} c_{i, \mu} c_{i, \nu} [\mu  \kappa| \nu  \lambda]\\
J_{\kappa \lambda} & = \sum_{i=1}^N  [\phi_i \kappa | \phi_i \lambda]
\end{split}
$$

In [3]:
def build_K(Fouridx,C,N):
    K = np.zeros(C.shape)
    for kappa in range(0,Fouridx.shape[0]):
        for lamda in range(0,Fouridx.shape[0]):
            K[kappa][lamda] = np.trace(C[:,0:N].T@Fouridx[:,kappa,:,lamda]@C[:,0:N])
    return 2*K # needs factor 2 because its spin restricted

In [4]:
mol = gto.Mole()
mol.atom = """
    Ne    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #mol.basis = "ccpvdz", mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 in the case of  case 16 distinct elements
eri = mol.intor('int2e')
eri.shape

(9, 9, 9, 9)

In [5]:
mf = scf.RHF(mol)
mf.kernel()
J = mf.get_j()
K = mf.get_k()
C = mf.mo_coeff
N = mol.nelec[0]
P=np.matmul(C[:,0:N],C[:,0:N].T)

converged SCF energy = -128.473876870668


In [6]:
print(J)

[[ 2.34250003e+01  4.30815331e+00  3.26823406e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 4.30815331e+00  1.24357919e+01  8.60782026e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 3.26823406e+00  8.60782026e+00  8.83395486e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.28730330e+01
   2.57074685e-18 -1.09724227e-18  4.93165923e+00  6.11710021e-18
  -5.65309458e-19]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.57074685e-18
   1.28730330e+01  6.82342258e-17  6.11710021e-18  4.93165923e+00
   2.25076000e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.09724227e-18
   6.82342258e-17  1.28730330e+01 -5.65309458e-19  2.25076000e-17
   4.93165923e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.93165923e+00
   6.11710021e-18 -5.6530945

In [7]:
J_2 = build_J(eri,C,N)
print(J_2)

[[ 2.34250003e+01  4.30815331e+00  3.26823406e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 4.30815331e+00  1.24357919e+01  8.60782026e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 3.26823406e+00  8.60782026e+00  8.83395486e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.28730330e+01
   6.07153217e-18  0.00000000e+00  4.93165923e+00  4.55364912e-18
  -8.67361738e-19]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.07153217e-18
   1.28730330e+01  8.32667268e-17  4.55364912e-18  4.93165923e+00
   2.08166817e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   8.32667268e-17  1.28730330e+01 -8.67361738e-19  2.08166817e-17
   4.93165923e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.93165923e+00
   6.07153217e-18 -8.6736173

In [8]:
print(J[0,0]/J_2[0,0], J[1,1]/J_2[1,1])

0.9999999999999999 1.0


In [9]:
print(K)

[[ 1.24802204e+01  2.01345026e+00  1.64951921e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.01345026e+00  3.65352126e+00  2.98783781e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 1.64951921e+00  2.98783781e+00  2.64819848e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.08225045e+00
  -8.02941674e-18 -8.84106684e-18  1.68027926e+00  6.64160195e-17
  -6.24887079e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.02941674e-18
   3.08225045e+00  6.33881006e-16  1.36763901e-16  1.68027926e+00
   1.91944590e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.84106684e-18
   6.33881006e-16  3.08225045e+00 -7.99630309e-18  2.72119906e-16
   1.68027926e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68027926e+00
   1.36763901e-16 -7.9963030

In [10]:
K_2 = build_K(eri,C,N)
print(K_2)

[[ 1.24802204e+01  2.01345026e+00  1.64951921e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.01345026e+00  3.65352126e+00  2.98783781e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 1.64951921e+00  2.98783781e+00  2.64819848e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.08225045e+00
   0.00000000e+00  2.77555756e-17  1.68027926e+00  5.89805982e-17
  -6.93889390e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   3.08225045e+00  5.55111512e-16  1.45716772e-16  1.68027926e+00
   1.66533454e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.38777878e-17
   5.55111512e-16  3.08225045e+00 -6.93889390e-18  2.22044605e-16
   1.68027926e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68027926e+00
   1.66533454e-16 -6.9388939